Bokeh is a python library for interactive visualization. Besides traditional graph tools, tt has some GIS capabilities

In [58]:
from bokeh.io import output_notebook, show, curdoc, push_notebook
from bokeh.plotting import Figure, ColumnDataSource
from bokeh.models import HoverTool, Select
from bokeh.tile_providers import WMTSTileSource
from bokeh.tile_providers import CARTODBPOSITRON_RETINA
from bokeh.layouts import layout, widgetbox
import numpy as np
import pandas as pd

In [59]:
def wgs84_to_web_mercator(lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    x = np.array(lon) * (k * np.pi/180.0)
    y = np.log(np.tan((90 + np.array(lat)) * np.pi/360.0)) * k
    return x,y

In [60]:
output_notebook()

Loading BokehJS ...

In [61]:
#map range
x_range,y_range = wgs84_to_web_mercator([-145,-45],[45,55])
x_range = tuple(x_range)
y_range = tuple(y_range)

fig = Figure(tools='pan, wheel_zoom, box_zoom, reset', x_range=x_range, y_range=y_range)
fig.axis.visible = False
tile = WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg')
fig.add_tile(tile)
show(fig)

In [62]:
fig = Figure(tools='pan, wheel_zoom, box_zoom, reset', x_range=x_range, y_range=y_range)
fig.axis.visible = False
fig.add_tile(CARTODBPOSITRON_RETINA)
show(fig)

Importing data from the kaggle Traffic Fatalities dataset

In [63]:
df = pd.read_csv("~/projects/kaggle_datasets/accident.csv")
df = df[(df["LONGITUD"]!=999.9999) & (df["MINUTE"]!=99)]     #removing missing data
df = df[["STATE","PERSONS","DAY","MONTH","YEAR","DAY_WEEK","HOUR","MINUTE","LATITUDE","LONGITUD","FATALS"]]
timestamp = zip(df["YEAR"],df["MONTH"],df["DAY"],df["HOUR"],df["MINUTE"])
df["timestamp"] = [str(x[0])+"/"+str(x[1])+"/"+str(x[2])+" "+str(x[3])+":"+str(x[4]) for x in list(timestamp)]

# January slice
jan = df[(df["MONTH"]==1)][["YEAR","DAY","MONTH","HOUR","MINUTE","timestamp","LONGITUD","LATITUDE","FATALS"]]

In [64]:
# convert to UTM
x_lon, y_lat = wgs84_to_web_mercator(jan["LONGITUD"],jan["LATITUDE"])

In [65]:
#prepare the plot
source = ColumnDataSource(data=dict(x=x_lon,y=y_lat,lon=jan["LONGITUD"], lat=jan["LATITUDE"], 
                                    fatals=jan["FATALS"],timestamp=jan["timestamp"],),)

In [66]:
# Make a new figure and add hover tool to the figure
fig = Figure(tools='pan, wheel_zoom, box_zoom, reset, hover', x_range=x_range, y_range=y_range)
hover = fig.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("timestamp", "@timestamp"),("Fatalities", "@fatals"), 
            ("longitude","@lon"),("latitude", "@lat")]

#plot data
fig.axis.visible = False
fig.add_tile(tile)
fig.circle(x="x", y="y", source=source, size="fatals", color="red", alpha=0.5)
fig.title.text = "January 2015 fatal accidents, USA"
show(fig)

Now creating a Select widget to choose the month to visualize

In [67]:
from ipywidgets import interact
#needed so taht callback functions can be called to execute calculations in the notebook

In [71]:
# convert to UTM
x_lon, y_lat = wgs84_to_web_mercator(df["LONGITUD"],df["LATITUDE"])
df["x_lon"]=x_lon; df["y_lat"]=y_lat

#prepare the plot
jan = df[df["MONTH"]==1]
source = ColumnDataSource(data=dict(x=jan["x_lon"],y=jan["y_lat"],lon=jan["LONGITUD"], lat=jan["LATITUDE"], 
                                    fatals=jan["FATALS"],timestamp=jan["timestamp"],),)

# Make a new figure and add hover tool to the figure
fig = Figure(tools='pan, wheel_zoom, box_zoom, reset, hover', x_range=x_range, y_range=y_range)
hover = fig.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("timestamp", "@timestamp"),("Fatalities", "@fatals"), 
            ("longitude","@lon"),("latitude", "@lat")]

#initialize select widget
months = ["January","February","March","April","May","June","July","August","September","October","November","December"]

#plot data
month_dic = {"January":1,"February":2,"March":3,"April":4,"May":5,"June":6,"July":7,"August":8,"September":9,"October":10,"November":11,"December":12}
fig.axis.visible = False
fig.add_tile(CARTODBPOSITRON_RETINA)
fig.circle(x="x", y="y", source=source, size="fatals", color="red", alpha=0.5)
fig.title.text = "{} 2015 fatal accidents, USA".format(month)

# Callback function
def update(month):
    #create new data
    df_new = df[(df["MONTH"]==month_dic[month])]
    source = ColumnDataSource(data=dict(x=df_new["x_lon"],y=df_new["y_lat"],lon=df_new["LONGITUD"], lat=df_new["LATITUDE"],
                                        fatals=df_new["FATALS"],timestamp=df_new["timestamp"],),)
    fig.title.text = "{} 2015 fatal accidents, USA".format(month)
    push_notebook()

show(fig, notebook_handle=True)


In [72]:
interact(update, month=months)

Now using <a href="http://datashader.readthedocs.io/en/latest/">datashader</a> to represent the large dataset of year long fatalities

In [70]:
#import datashader as ds
#import datashader.transfer_functions as tf